DATA FOR THE PUMP ARE CALCULATED FOR A SINGLE STAGE. MULTISTAGE DATA CAN BE CALCULATED FURTHER.

NOTE:<br>
1 metre = 3.281 feet <br>
1 litre = 0.001 cu.mts <br>
1 atm = 101325 Pa = 1.0133 bar <br>
1 kPa = 1000 Pa <br>
1 bar = 100 kPa = 0.1 MPa

VAPOUR PRESSURE DATA OF WATER FOR INTERPOLATION IN THE RANGE 0 TO 100 DEGREE CENTIGRADE: <br><br>
Temperature (degree C) - Vapour Pressure (kPa) <br><br>
0 - 0.611 <br>
0.01 - 0.612 (This is the triple point of water) <br>
05 - 0.873 <br>
10 - 1.228 <br>
15 - 1.705 <br>
20 - 2.339 <br>
25 - 3.169 <br>
30 - 4.245 <br>
35 - 5.627 <br>
40 - 7.381 <br>
45 - 9.590 <br>
50 - 12.345 <br>
55 - 15.752 <br>
60 - 19.933 <br>
65 - 25.024 <br>
70 - 31.177 <br>
75 - 38.565 <br>
80 - 47.375 <br>
85 - 57.817 <br>
90 - 70.120 <br>
95 - 84.532 <br>
100 - 101.325

DENSITY DATA OF WATER FOR INTERPOLATION IN THE RANGE 0 T0 100 DEGREE CENTIGRADE <BR>
Temperature (degree C) - Density (kg/m^3)<br>
0 - 999.87 <br>
4 - 1000 (Maximum density of water) <br>
5 - 999.99 <br>
10 - 999.73 <br>
15 - 999.13 <br>
20 - 998.23 <br>
25 - 997.07 <br>
30 - 995.62 <br>
40 - 992.24 <br>
50 - 988.07 <br>
60 - 983.24 <br>
70 - 977.81 <br>
80 - 971.83 <br>
90 - 965.34 <br>
100 - 958.23 

In [1]:
import math # Onboarding mathematical functions
from math import pi

In [8]:
# INLET FLUID DATA
g = 9.80665 # Acceleration due to gravity in m/s^2
N = float(input('Enter the pump rotation speed, in rpm: '))
d1 = float(input('Enter the pump inlet diameter, in cm: '))
d2 = float(input('Enter the pump outlet diameter, in cm: '))
h2 = float(input('Enter the impeller width at the outlet, in cm: '))
q = float(input('Enter the volume of fluid handled, in litres/sec: '))
Q = 0.001*q
u1 = (pi*d1*N)/6000
u2 = (pi*d2*N)/6000
p1 = float(input('Enter the asbolute pressure at the pump inlet, in kPa: '))
t1 = float(input('Enter the inlet temperature of the fluid, in degree C: '))
T1 = t1 + 273.15 # Conversion from Centigrade scale to Kelvin scale
rho = float(input('Enter the density of the fluid at the inlet temperature, in kg/m^3: '))
pv = float(input('Enter the vapour pressure of the fluid at the inlet temperature, in kPa: '))
head_check = input('Is the pump head specified?: ').upper()
if head_check == 'Y':
    Hm = float(input('Enter the head developed by the pump, in metres: '))
    Ns = (N*(math.sqrt(Q))/(Hm**0.75))
elif head_check == 'N':
    Ns = float(input('Enter the specific speed of the pump: '))
    Hm = (60*Ns/(N*math.sqrt(Q)))**(4/3)
b1d = float(input('Enter the inlet blade angle, in degrees: '))
b1 = b1d*pi/180
a1d = 90

Enter the pump rotation speed, in rpm: 1500
Enter the pump inlet diameter, in cm: 30
Enter the pump outlet diameter, in cm: 60
Enter the impeller width at the outlet, in cm: 5.5
Enter the volume of fluid handled, in litres/sec: 300
Enter the asbolute pressure at the pump inlet, in kPa: 101.325
Enter the inlet temperature of the fluid, in degree C: 20
Enter the density of the fluid at the inlet temperature, in kg/m^3: 998.23
Enter the vapour pressure of the fluid at the inlet temperature, in kPa: 2.339
Is the pump head specified?: y
Enter the head developed by the pump, in metres: 80
Enter the inlet blade angle, in degrees: 22


In [9]:
# INLET VELOCITY TRIANGLE
c1x = (u1*math.tan(b1))
c1 = c1x
w1 = math.hypot(c1x,u1)

In [22]:
# OUTLET VELOCITY TRIANGLE
b2d = float(input('Enter the exit blade angle, in degrees: '))
b2 = b2d*pi/180
c2x = 10000*Q/(pi*d2*h2)
c2u = u2 - (c2x/math.tan(b2))
c2 = math.hypot(c2x,c2u)
a2 = math.atan(c2x/c2u)
a2d = a2*180/pi
nmano = ((g*Hm)/(u2*c2u))*100
nmech = float(input('Enter the desired mechanical efficiency of the drive, in percentage: '))
noverall = (nmano*nmech)*0.01
Pshaft = ((100*rho*g*Q*Hm)/(noverall))*0.001
Torque = 60000*Pshaft/(2*pi*N)

Enter the exit blade angle, in degrees: 28
Enter the desired mechanical efficiency of the drive, in percentage: 80


In [23]:
# EVALUATING FOR CAVITATION
sigma_critical = 0.00103*((Ns)**4/3)
NPSHa = ((1000*(p1-pv))+(c1**2))/(rho*g)
sigma = NPSHa/Hm
dp = (((c1**2)-(c2**2)+(u2*c2u))/(2*g))

In [24]:
# MULTISTAGING OF THE PUMP
multistage_check = input('Is the pump mult-staged?: ').upper()
if multistage_check == 'Y':
    series_parallel_check = input('Is the multi-staging in series or parallel?: ').upper()
    if series_parallel_check == 'S':
        stage_check = input('Do you know the number of stages?: ').upper()
        if stage_check == 'Y':
            stages = int(input('Enter the number of stages: '))
            Ht = stages*Hm
            qt = q
        elif stage_check == 'N':
            Ht = float(input('Enter the total head developed in multistaging, in metres: '))
            stages = Ht/Hm
            qt = q
    elif series_parallel_check == 'P':
        stage_check = input('Do you know the number of stages?: ').upper()
        if stage_check == 'Y':
            stages = int(input('Enter the number of stages: '))
            qt = stages*q
            Ht = Hm
        elif stage_check == 'N':
            qt = float(input('Enter the total volume of fluid handled in multi-staging, in litres/sec: '))
            stages = qt/q
            Ht = Hm
elif multistage_check == 'N':
    stages = 1
    Ht = Hm
    qt = q

Is the pump mult-staged?: n


In [25]:
# PRINTING THE RESULTS
print('THE FOLLOWING DATA ARE EVALUATED FOR THE GIVEN CENTRIFUGAL PUMP:\n')
print(f'1. The inlet flow and blade angles are {a1d} and {b1d} respectively.')
print(f'2. The exit flow and blade angles are {a2d} and {b2d} respectively.')
print(f'3. The single stage manometric head of the pump is {Hm} metres.')
print(f'4. The single stage pump discharge is {q} litres/sec.')
print(f'5. The specific speed of the pump is {Ns}.')
print(f'6. The manometric efficiency of the pump is {nmano} %.')
print(f'7. The overall efficiency of the pump is {noverall} %.')
print(f'8. The power available at the pump shaft is {Pshaft} kW.')
print(f'9. The torque acting on the shaft is {Torque} N-m')
print(f'10. The NPSHa of the pump is {NPSHa} metres.')
print(f'11. The pressure rise in the impeller is {dp} kPa.')
print(f'12. The total head of the {stages}-stage pump is {Ht} metres')
print(f'13. The total discharge of the {stages}-stage pump is {qt} litres/s')
if sigma < sigma_critical:
    print('14. No cavitation in the pump.')
elif sigma == sigma_critical:
    print('14. Cavitation just begins. Onset of bubbling.')
elif sigma > sigma_critical:
    print('14. Warning!!! Cavitation has occurred. Blade pitting and erosion taking place.')

THE FOLLOWING DATA ARE EVALUATED FOR THE GIVEN CENTRIFUGAL PUMP:

1. The inlet flow and blade angles are 90 and 22.0 respectively.
2. The exit flow and blade angles are 3.971363108042085 and 28.0 respectively.
3. The single stage manometric head of the pump is 80.0 metres.
4. The single stage pump discharge is 300.0 litres/sec.
5. The specific speed of the pump is 30.713859705477.
6. The manometric efficiency of the pump is 39.94158857709768 %.
7. The overall efficiency of the pump is 31.95327086167815 %.
8. The power available at the pump shaft is 735.2706222941619 kW.
9. The torque acting on the shaft is 4680.878161934792 N-m
10. The NPSHa of the pump is 10.12091796760096 metres.
11. The pressure rise in the impeller is 15.759405749527623 kPa.
12. The total head of the 1-stage pump is 80.0 metres
13. The total discharge of the 1-stage pump is 300.0 litres/s
14. No cavitation in the pump.
